In [10]:
import warnings
import requests
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup


# wanring 메시지 출력 안함
warnings.filterwarnings('ignore')

# 최신 회차 크롤링 함수
def get_max_count():
    url = 'https://dhlottery.co.kr/common.do?method=main'
    html = requests.get(url).text
    headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"}
    r=requests.get(url,headers=headers)
    soup=BeautifulSoup(r.content,'html.parser')
    max_count = int(soup.find('strong', id='lottoDrwNo').text)
    return max_count

# 로또 당첨번호 정보 조회 함수
def crawling_lotto(count):
    # url에 회차를 실어 페이지 조회
    url = f'https://dhlottery.co.kr/gameResult.do?method=byWin&drwNo={count}'
    headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"}
    # url="https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"
    r=requests.get(url,headers=headers)
    # html = requests.get(url).text
    soup=BeautifulSoup(r.content,'html.parser')
    
    date = datetime.strptime(soup.find('p', class_='desc').text, '(%Y년 %m월 %d일 추첨)')
    win_number = [int(i) for i in soup.find('div', class_='num win').find('p').text.strip().split('\n')]
    bonus_number = int(soup.find('div', class_='num bonus').find('p').text.strip())
    
    return {
        'date': date, 
        'win_number': win_number, 
        'bonus_number': bonus_number
    }

# 최신 회차 가져오기
max_count = get_max_count()
# 전체 회차 크롤링
data = pd.DataFrame()
list=[]
for i in tqdm(range(500, max_count+1)):
    result = crawling_lotto(i)
    data = data._append({'date': result['date'],
                        'num1': result['win_number'][0],
                        'num2': result['win_number'][1],
                        'num3': result['win_number'][2],
                        'num4': result['win_number'][3],
                        'num5': result['win_number'][4],
                        'num6': result['win_number'][5],
                        'bonus': result['bonus_number'],
                       }, ignore_index=True)
data

  0%|          | 0/611 [00:00<?, ?it/s]

,date,num1,num2,num3,num4,num5,num6,bonus
0,2012-06-30,3,4,12,20,24,34,41
1,2012-07-07,1,4,10,17,31,42,2
2,2012-07-14,6,22,28,32,34,40,26
3,2012-07-21,1,5,27,30,34,36,40
4,2012-07-28,6,14,22,26,43,44,31
...,...,...,...,...,...,...,...,...
606,2024-02-10,1,3,4,29,42,45,36
607,2024-02-17,6,14,30,31,40,41,29
608,2024-02-24,7,19,26,37,39,44,27
609,2024-03-02,10,12,13,19,33,40,2


In [14]:
data['num1']

0       3
1       1
2       6
3       1
4       6
       ..
606     1
607     6
608     7
609    10
610     3
Name: num1, Length: 611, dtype: int64

In [15]:
from collections import Counter

In [18]:
number_counts1 = Counter(data['num1'])
number_counts2 = Counter(data['num2'])
number_counts3 = Counter(data['num3'])
number_counts4 = Counter(data['num4'])
number_counts5 = Counter(data['num5'])
number_counts6 = Counter(data['num6'])
number_counts7 = Counter(data['bonus'])



In [23]:
# max_num1 = max(number_counts1, key=number_counts1.get)
# max_num2 = max(number_counts2, key=number_counts2.get)
# max_num3 = max(number_counts3, key=number_counts3.get)
# max_num4 = max(number_counts4, key=number_counts4.get)
# max_num5 = max(number_counts5, key=number_counts5.get)
# max_num6 = max(number_counts6, key=number_counts6.get)
# max_bonus = max(number_counts7, key=number_counts7.get)
# 
# # 각 필드에서 앞에서 찾은 숫자가 나타난 횟수를 찾습니다.
# max_num1_count = number_counts1[max_num1]
# max_num2_count = number_counts2[max_num2]
# max_num3_count = number_counts3[max_num3]
# max_num4_count = number_counts4[max_num4]
# max_num5_count = number_counts5[max_num5]
# max_num6_count = number_counts6[max_num6]
# max_bonus_count = number_counts7[max_bonus]
# 
# # 이러한 정보들로 구성된 데이터프레임을 만듭니다.
# df_final = pd.DataFrame({
#     'Number': ['num1', 'num2', 'num3', 'num4', 'num5', 'num6', 'bonus'],
#     'Max_Number': [max_num1, max_num2, max_num3, max_num4, max_num5, max_num6, max_bonus],
#     'Count': [max_num1_count, max_num2_count, max_num3_count, max_num4_count, max_num5_count, max_num6_count, max_bonus_count]
# })
# df_final

,Number,Max_Number,Count
0,num1,1,78
1,num2,7,45
2,num3,13,38
3,num4,26,33
4,num5,34,53
5,num6,45,88
6,bonus,12,19


In [30]:
# 각 필드에서 두 번째로 빈번하게 나타난 숫자를 찾습니다.
num1=[]
num2=[]
num3=[]
num4=[]
num5=[]
num6=[]
bon=[]
for i in range(1,10):
  num1.append(number_counts1.most_common(i)[-1][0])
  num2.append(number_counts2.most_common(i)[-1][0])
  num3.append(number_counts3.most_common(i)[-1][0])
  num4.append(number_counts4.most_common(i)[-1][0])
  num5.append(number_counts5.most_common(i)[-1][0])
  num6.append(number_counts6.most_common(i)[-1][0])
  bon.append(number_counts7.most_common(i)[-1][0])
  # second_max_num1 = number_counts1.most_common(i)[-1]
  # second_max_num2 = number_counts2.most_common(i)[-1]
  # second_max_num3 = number_counts3.most_common(i)[-1]
  # second_max_num4 = number_counts4.most_common(i)[-1]
  # second_max_num5 = number_counts5.most_common(i)[-1]
  # second_max_num6 = number_counts6.most_common(i)[-1]
  # second_max_bonus = number_counts7.most_common(i)[-1]

# 찾은 숫자와 빈도수를 데이터 프레임으로 만듭니다.
#   df_second_final = pd.DataFrame({
#     'Number': ['num1', 'num2', 'num3', 'num4', 'num5', 'num6', 'bonus'],
#     'Second_Max_Number': [second_max_num1[0], second_max_num2[0], second_max_num3[0], second_max_num4[0], second_max_num5[0], second_max_num6[0], second_max_bonus[0]],
#   })
lottodf=pd.DataFrame({"num1":num1,"num2":num2,"num3":num3,"num4":num4,"num5":num5,"num6":num6,"bon":bon})
lottodf

,num1,num2,num3,num4,num5,num6,bon
0,1,7,13,26,34,45,12
1,2,10,18,24,39,44,32
2,3,12,19,33,38,43,24
3,6,8,12,23,33,42,30
4,4,18,16,31,40,41,6
5,5,11,20,30,36,40,2
6,7,6,17,32,37,39,31
7,8,14,22,27,35,38,4
8,12,15,15,21,42,36,10
